# Create Result Tables

In [48]:
import glob
import pathlib
import json
import pandas as pd
from timeslib.reporting import print_to_xlsx
from timeslib.misc import read_data_csv

### Specify input folder

In [49]:
input_folder = "./input/"

### Specify output folder

In [50]:
output_folder = "./output/"

### Load table info

In [51]:
with open("./tim-tables-info/table_info.json", "r") as file:
    table_info = json.load(file)

In [52]:
# get list of all input data files with a certain name extension
path_list = glob.glob(input_folder + "*.csv")
print(
    "Found {} csv files.\n".format(len(path_list)),
    "\n".join("{}".format(k) for k in path_list),
    sep="",
)

Found 119 csv files.
./input\AGR_FEC.csv
./input\AGR_FuelPrice.csv
./input\AGR_NRG-BIO-PRIME.csv
./input\AGR_NRG-BIO-PRIME_SEAI.csv
./input\IND-CAF_FEC.csv
./input\IND-CON_FEC.csv
./input\IND-EOE_FEC.csv
./input\IND-FAB_FEC.csv
./input\IND-MAE_FEC.csv
./input\IND-MAP_FEC.csv
./input\IND-NEM_FEC.csv
./input\IND-OMA_FEC.csv
./input\IND-ONM_FEC.csv
./input\IND-PX4_FEC.csv
./input\IND-RAP_FEC.csv
./input\IND-TAP_FEC.csv
./input\IND-TEM_FEC.csv
./input\IND-WAP_FEC.csv
./input\IND_Emissions-CO2.csv
./input\IND_FEC.csv
./input\IND_FEC_Sector.csv
./input\IND_FuelPrice.csv
./input\PWR_Cap-N.csv
./input\PWR_Cap.csv
./input\PWR_Emissions-CO2.csv
./input\PWR_FuelPrice.csv
./input\PWR_Gen-ELCC.csv
./input\PWR_Gen-HETC.csv
./input\Residential Services.csv
./input\RSD_BLD-N_TYPE.csv
./input\RSD_BLD_TYPE.csv
./input\RSD_CK_TYPE.csv
./input\RSD_FEC.csv
./input\RSD_FuelPrice.csv
./input\RSD_OtherServices_FuelCons.csv
./input\RSD_RTFT-ATT_NCAP.csv
./input\RSD_RTFT-DET_NCAP.csv
./input\RSD_RTFT_NRG_SAVING

### Read csv data into a dataframe

In [53]:
# Create an empty DataFrame
data = pd.DataFrame()
# Read data into the dataframe
for a_table in table_info.keys():
    for a_table_rule in table_info[a_table].keys():
        file_path = glob.glob(input_folder + a_table_rule + ".csv")
        if file_path:
            df = read_data_csv(file_path[0],
                               {a_table_rule: table_info[a_table][a_table_rule]})
            if df is not None:
                df["tableName"] = a_table
                data = data.append(df, ignore_index=True)

assert len(data.index), "The dataframe is empty. No data has been read."

data = data.groupby([i for i in data.columns if not i == "total"]).agg("sum")
data = data.reset_index()

### Print results to excel files

In [57]:
for aScenario in data["scenario"].unique():
        temp_df = (
            data[
                (data["scenario"] == aScenario)
            ]
            .drop(columns=["scenario"])
            .pivot(index=["tableName", "serieName"], columns="year", values="total")
        )
        temp_df.fillna(value=0, inplace=True)
        temp_df.index = [temp_df.index.map("{0[0]} [{0[1]}]".format)]
        temp_df.to_excel(output_folder + aScenario + ".xlsx")